# The Inverse Problem
The inverse problem is the problem of finding the solution to a system of linear equations of the form :

Given $f$ let's find $\psi$ such that $$f=curl(\psi)$$ , and let $$\psi\vert_{\partial\Omega}=\psi_0$$

Obvuoisly we need to define some boundary conditions for $\psi$.



In [8]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
from ngsolve.krylovspace import CGSolver


import scipy.sparse as sp
from scipy.sparse import csc_matrix
import matplotlib.pylab as plt

from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
import matplotlib.pylab as plt
import scipy.sparse as sp
from scipy.sparse.linalg import inv as inv
from scipy.sparse import csc_matrix


In [9]:
n = specialcf.normal(3) # normal vector to a surface of a tetrahedron

def norm(u, Mesh):
    # norm of a scalar, vector or matrix function
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh)) 

def Curl(u):
    # curl of a vector or matrix
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )

def Inc(u):
    # inc of a matrix
    return Curl((Curl(u)).trans)

P_n = OuterProduct(n,n)  # projection along normal direction
Q_n = Id(3) - OuterProduct(n,n) # projection along tangential direction
C_n = CF( (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) ) # corss product perator with the normal vector

def C(V): 
    # corss product perator with the generic vector
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 


In [16]:


def distributive_inverse_curl(mesh,order = 1,  BND_psi = None, curl_psi = None ,draw = False, bonus_intorder = 15):

    if order < 1:
        raise Exception("Wrong order!")

    fes = HCurl(mesh, order=order, dirichlet=".*")*H1(mesh,order=order+1)#, dirichlet=".*")
    (u,p), (v,q) = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True, symmetric_storage=True, condense=False)
    a += (curl(u)*curl(v) + Grad(p)*v + Grad(q)*u +10**(-10)*u*v )*dx

    f = LinearForm(fes)
    f += curl_psi*curl(v)*dx

    a.Assemble()
    f.Assemble()

    gf_sol = GridFunction(fes)
    gf_psi, gf_p = gf_sol.components

    gf_psi.Set(BND_psi, definedon=mesh.Boundaries(".*"),  bonus_intorder = bonus_intorder, dual=True)
    
    r = f.vec.CreateVector()
    #inv = a.mat.Inverse(fes.FreeDofs(a.condense), inverse="pardiso")
    r.data = f.vec - a.mat * gf_sol.vec   
    gf_sol.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso") * r
    gf_psi , gf_p = gf.components

    #r.data = f.vec
    #if a.condense:
    #    r.data += a.harmonic_extension_trans * r
    #w.data = inv * r
    #if a.condense:
    #    w.data += a.harmonic_extension * w
    #    w.data += a.inner_solve * r
    #gf_sol.vec.data = w

    gf_curl_psi = CF( (Grad(gf_psi)[2,1]-Grad(gf_psi)[1,2],Grad(gf_psi)[0,2]-Grad(gf_psi)[2,0],Grad(gf_psi)[1,0]-Grad(gf_psi)[0,1]) )
     
    return gf_psi, gf_curl_psi



In [17]:

peak = exp(-25*((x-0.5)**2+(y-0.5)**2+(z-0.5)**2))

# our initail function
psi = CF ( (peak, 0, 0 ))#, dims=(3,1))

# curl that needs to be approx
curl_psi = CF( Curl(psi))#, dims=(3,1) )

Maxh = 0.1
Order = 3

with TaskManager():
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    gf_psi, gf_curl_psi =  distributive_inverse_curl(mesh,order = Order,  BND_psi = psi, curl_psi = curl_psi ,draw = False)

draw = True
if draw:
    print("curl_psi : ")
    Draw(curl_psi, mesh,clipping = (1,1,1,0))
    print("gf_curl_psi : ")
    Draw(gf_curl_psi, mesh,clipping = (1,1,1,0))
    print("Error gf_curl_psi -curl_psi "+ str(sqrt(Integrate(InnerProduct(gf_curl_psi -curl_psi,gf_curl_psi -curl_psi),mesh))))
    Draw(gf_curl_psi -curl_psi, mesh,clipping = (1,1,1,0))

    print("psi : ")
    Draw(psi, mesh, clipping = (1,1,1))
    print("gf_psi : ")
    Draw(gf_psi, mesh,clipping = (1,1,1,0))
    print("Error psi-gf_psi "+ str(sqrt(Integrate(InnerProduct(psi-gf_psi,psi-gf_psi),mesh))))
    Draw(psi-gf_psi, mesh,clipping = (1,1,1,0))



NameError: name 'gf' is not defined